In [11]:
import pandas as pd
import geopy
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import regex as re
locator = Nominatim(user_agent = 'myGeoencoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds = 1)
addresses = pd.read_csv('C:/Users/barre/Downloads/uk_total.csv')

In [13]:
addresses['location'] = addresses['address'].apply(geocode)
addresses['point'] = addresses['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [15]:
addresses_all = pd.read_csv('C:/Users/barre/Downloads/nodes-addresses.csv')

In [17]:
addresses_all['country_codes'] = addresses_all['country_codes'].astype(str)
addresses_uk = addresses_all[addresses_all['country_codes'].str.contains('GBR')]

In [18]:
addresses_uk['location'] = addresses_uk['address'].apply(geocode)
addresses_uk['point'] = addresses_uk['location'].apply(lambda loc: tuple(loc.point) if loc else None)

<ipython-input-18-db3b2428e498>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_uk['location'] = addresses_uk['address'].apply(geocode)
<ipython-input-18-db3b2428e498>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_uk['point'] = addresses_uk['location'].apply(lambda loc: tuple(loc.point) if loc else None)


In [19]:
print(addresses_uk.shape)
print(addresses.shape)

(977, 11)
(4216, 17)


In [20]:
addresses = addresses[addresses.node_id.isin(addresses_uk.node_id)]

In [21]:
print(sum(addresses_uk['point'].value_counts()))
print(sum(addresses['point'].value_counts()))
addresses['point_alt'] = addresses['point']

333
329


In [22]:
postcodes = pd.read_csv('C:/Users/barre/Downloads/postcodes/postcodes.csv')
addresses_uk["PostCode"] = addresses_uk["address"].str.extract(r"(\b(?:[A-Za-z][A-HJ-Ya-hj-y]?[0-9][0-9A-Za-z]? ?[0-9][A-Za-z]{2}|[Gg][Ii][Rr] ?0[Aa]{2})\b)")

<ipython-input-22-11473b6ad77b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_uk["PostCode"] = addresses_uk["address"].str.extract(r"(\b(?:[A-Za-z][A-HJ-Ya-hj-y]?[0-9][0-9A-Za-z]? ?[0-9][A-Za-z]{2}|[Gg][Ii][Rr] ?0[Aa]{2})\b)")


In [23]:
addresses_uk_final = pd.merge(addresses_uk, addresses['point_alt'], how = 'left', left_index = True, right_index = True)

In [24]:
addresses_uk_final = pd.merge(addresses_uk_final, postcodes[['Postcode', 'Latitude', 'Longitude']], how = 'left', left_on = 'PostCode', right_on = 'Postcode')

In [55]:
addresses_uk_final['postcode_latlong'] = list(zip(addresses_uk_final['Latitude'], addresses_uk_final['Longitude']))
addresses_uk_final['postcode_latlong'] = addresses_uk_final['postcode_latlong'].map(lambda x: None if str(x) == '(nan, nan)' else x)
addresses_uk_final[['postcode_latlong', 'point', 'point_alt']] = addresses_uk_final[['postcode_latlong', 'point', 'point_alt']].fillna('No Address Found')
addresses_uk_final['latlong_final'] = addresses_uk_final[['postcode_latlong', 'point', 'point_alt']].apply(lambda x: x['postcode_latlong'] if x['postcode_latlong'] != 'No Address Found' else x['point'] if x['point'] != 'No Address Found' else x['point_alt'], axis = 1)


In [57]:
addresses_uk_final.to_csv('C:/Users/barre/Documents/addresses_uk_fixed.csv')

In [29]:
addresses_uk_final['postcode_latlong'].value_counts().sum()

568

In [32]:
addresses_uk['PostCode'].value_counts().sum()

628

In [45]:
addresses_uk[addresses_uk['PostCode'].isna()]['address']

360      FLAT 1184, 6 SLINGTON HOUSE, RANKINE ROAD, BAS...
659            27 GLOUCESTER PLACE, LONDON, UNITED KINGDOM
736      18 NEW ROAD, HYTHE, SOUTHAMPTON, S045 6BP HAMP...
738      18 NEW ROAD, HYTHE, SOUTHAMPTON S045 6BP HAMPS...
811      53A SAILMAKERS COURT, SW6 2 UX, LONDON, UNITED...
                               ...                        
15561                                  17 TOFTENHAM STREET
15563                   WILLARDS FARM, THE COMMON DUNSFOLD
15564                            13 HAMPSTEAD HILL GARDENS
15565                                 25 ST  PETERS SQUARE
15818    19, PENTHOUSE NORTH, 12A-I2J ENNISMORE GARDENS...
Name: address, Length: 349, dtype: object

In [46]:
for i in addresses_uk[addresses_uk['PostCode'].isna()]['address']:
    print(i)

FLAT 1184, 6 SLINGTON HOUSE, RANKINE ROAD, BASINGSTOKE RG24 8 PH, HAMPSHIRE, UNITED KINGDOM
27 GLOUCESTER PLACE, LONDON, UNITED KINGDOM
18 NEW ROAD, HYTHE, SOUTHAMPTON, S045 6BP HAMPSHIRE, UNITED KINGDOM
18 NEW ROAD, HYTHE, SOUTHAMPTON S045 6BP HAMPSHIRE
53A SAILMAKERS COURT, SW6 2 UX, LONDON, UNITED KINGDOM
SERENDIPITY, MAINSTREET, MAIDS MORETON MK18 71QS
17 AGATE ROAD, W6 OAJ LONDON, UNITED KINGDOM
52 QUEENSMERE ROAD, LONDON, WIMBLEDON, UNITED KINGDOM
3 EAST MOOR DRIVER, WOLVERTON MILL, MILTON KEYNES, BUCKINGHAMSHIRE, UNITED KINGDOM
52 QUEENSMERE ROAD, LONDON, WIMBLEDON, UNITED KINGDON
15 LANGWOOD GARDENS, WATFORD, HERTFORDSHIRE, UNITED KINGDOM
15 LANGWOOD GARDENS, WATFORD, HERTFORSHINE, UNITED KINGDOM
FLAT 4, GORDON HOUSE, 37 WELBECK STREET, LONDON W1G 8 DW, UNITED KINGDOM
37 WELBECK STRET, LONDON 7063
FLAT 1, 57 CADOGAN GARDENS, LONDON SW3 2 TH, LONDON, UNITED KINGDOM
115B DRYSDALE STREET, LONDON, UNITED KINGDOM
FLAT C, 46-48, GERTRUDES STREET, LONDON, UNITED KINGDOM
URB. SORRIBES 